In [1]:
import os
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]
model = "gpt-4o-mini"

In [2]:
from openai import OpenAI

client = OpenAI()

In [3]:
messages = [
    {"role": "system", "content": (
        "You are a fact checker. Verify the validity of the sentence provided by the user, given a reference. You must return a response in JSON format:"
        "{'isFactTrue': <true or false>, 'explanation': <explanation> ")
    },
    {"role": "user", "content": "White sand is made from fish poop https://oceanservice.noaa.gov/facts/sand.html"}
]

In [4]:
def get_text_from_url(url):
    return '''The famous white-sand beaches of Hawaii, for example, actually come from the poop of parrotfish. The fish bite and scrape algae off of rocks and dead corals with their parrot-like beaks, grind up the inedible calcium-carbonate reef material (made mostly of coral skeletons) in their guts, and then excrete it as sand.'''

In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_text_from_url",
            "description": "Get the text contents of a url",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "The url",
                    },
                },
                "required": ["url"],
            },
        },
    }
]

response = client.chat.completions.create(
  model=model,
  messages=messages,
  tools=tools,
  response_format={"type": "json_object"},
)

In [7]:
import json

def process_response_with_tools(response):
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        available_functions = {
            "get_text_from_url": get_text_from_url,
        } 
        messages.append(response_message) 
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                url=function_args.get("url"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            ) 
        return client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        ) 
    
response_after_tool = process_response_with_tools(response)
messages.append({"role": "assistant", "content": response_after_tool.choices[0].message.content})

In [8]:
while True:
    user_input = input("Please enter your input: ")
 
    if user_input.lower() == 'exit':
        break

    messages.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    ) 

    print(response.choices[0].message.content)

    messages.append({"role": "assistant", "content": response.choices[0].message.content})


{"isFactTrue":false,"explanation":"While some white sand does originate from the excrement of parrotfish, it is not accurate to state that all white sand is made from fish poop. White sand can also come from other sources, such as the erosion of coral and shells. Therefore, the statement is an oversimplification."}
{"isFactTrue":null,"explanation":"No additional information needed. Exiting the conversation."}
